In [4]:
# Cell 1 — Import
import json
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go

from gini_paris_distances_calculations import (
    build_graph_from_edgelist,
    build_node_index,
    accessibility_inequality_to_targets,
)

def round_minutes(x) -> int:
    try:
        v = float(x)
    except Exception:
        return 0
    if not np.isfinite(v) or v <= 0:
        return 0
    return int(math.floor(v + 0.5))

def fmt_min(x) -> str:
    return str(round_minutes(x))


In [5]:
# Cell 2 — Caricamento dati (stessi nomi file dello Streamlit)

G = build_graph_from_edgelist("./timed_edgelist.geojson")
node_index = build_node_index(G)

centers_gdf = gpd.read_file("./centri_esagoni_parigi.geojson")
hexes_gdf = gpd.read_file("./esagoni_parigi.geojson")

# labels opzionali
try:
    with open("./id_to_labels_centers.json", "r", encoding="utf-8") as f:
        center_labels = json.load(f)
    if not isinstance(center_labels, dict):
        center_labels = {}
except Exception:
    center_labels = {}

# targets = centri esagoni (lon, lat)
targets_lonlat = [(geom.x, geom.y) for geom in centers_gdf.geometry]
len(targets_lonlat), hexes_gdf.shape


(653, (653, 4))

In [6]:
# Cell 3 — INSERISCI QUI I DUE PUNTI (lon, lat) (WGS84 / EPSG:4326)
# Esempi (da sostituire):
start_a = (2.3730, 48.8462)  # lon, lat
start_b = (2.2945, 48.8584)  # lon, lat

starts_lonlat = [start_a, start_b]
starts_lonlat


[(2.373, 48.8462), (2.2945, 48.8584)]

In [7]:
# Cell 4 — Calcolo metriche (Gini per ogni target = esagono)
# Parametri identici (o quasi) allo Streamlit
max_line_changes = 1
change_penalty_min = 2.0

metrics_df = accessibility_inequality_to_targets(
    G,
    starts_lonlat=starts_lonlat,
    targets_lonlat=targets_lonlat,
    node_index=node_index,
    max_line_changes=max_line_changes,
    change_penalty_min=change_penalty_min,
    max_walk_min_start=15.0,
    max_walk_min_end=15.0,
    max_candidate_stations=25,
    allow_walk_only=True,
    keep_details=False,
    return_per_target_df=False
)

metrics_df.head()


,n_total,n_ok,share_ok,mean_time_min,median_time_min,p90_time_min,min_time_min,max_time_min,gini_time,theil_time,target_id,target_lon,target_lat,mean_time_softmax,gini_time_norm
0,2,2,1.0,33.606498,33.606498,34.516746,32.468688,34.744308,8.975894e-19,0.000573,0,2.330755,48.818873,2.651132e-17,0.033857
1,2,2,1.0,38.455974,38.455974,39.366221,37.318164,39.593783,1.001469e-16,0.000438,1,2.336652,48.818906,3.384791e-15,0.029587
2,2,2,1.0,36.784616,36.784616,46.101482,25.138534,48.430698,2.014578e-16,0.050991,2,2.342549,48.818939,6.363127e-16,0.316602
3,2,2,1.0,29.243726,29.243726,36.385330,20.316720,38.170732,1.031280e-19,0.047345,3,2.348446,48.818971,3.378340e-19,0.305262
4,2,2,1.0,23.674459,23.674459,30.816064,14.747453,32.601465,4.857664e-22,0.072881,4,2.354343,48.819003,1.288255e-21,0.377073


In [8]:
# Cell 5 — Funzioni Plotly per mappa (derivate dal tuo Streamlit)

def gini_to_color_hex(v):
    v = float(np.clip(v, 0.0, 1.0))
    blue = np.array([59, 130, 246])   # #3b82f6
    green = np.array([34, 197, 94])   # #22c55e
    amber = np.array([245, 158, 11])  # #f59e0b
    red = np.array([239, 68, 68])     # #ef4444

    if v <= 0.1:
        t = v / 0.1 if 0.1 else 0
        rgb = blue + (green - blue) * t
    elif v <= 0.55:
        t = (v - 0.1) / (0.55 - 0.1)
        rgb = green + (amber - green) * t
    else:
        t = (v - 0.55) / (1 - 0.55)
        rgb = amber + (red - amber) * t

    rgb = np.clip(rgb.round().astype(int), 0, 255)
    return "#{:02x}{:02x}{:02x}".format(rgb[0], rgb[1], rgb[2])

def _hex_to_rgba(h, a):
    h = str(h).lstrip("#")
    if len(h) != 6:
        return f"rgba(200,200,200,{a})"
    r = int(h[0:2], 16)
    g = int(h[2:4], 16)
    b = int(h[4:6], 16)
    return f"rgba({r},{g},{b},{a})"

def render_hexagon_heatmap(hexes_gdf, metrics_df, labels_map=None, gini_col="gini_time_norm"):
    """
    Mappa Plotly con esagoni colorati in base al Gini.
    - hexes_gdf deve avere colonna 'id' coerente con metrics_df.target_id
    - gini_col: di default 'gini_time_norm' (Gini "puro" 0..1)
      (nel tuo codice esiste anche 'gini_time' che è quello pesato dalla softmax)
    """
    labels_map = labels_map or {}

    # join
    cols = ["target_id", gini_col, "mean_time_min"]
    hex_merged = hexes_gdf.merge(
        metrics_df[cols],
        left_on="id",
        right_on="target_id",
        how="left"
    )

    # WGS84 per mapbox
    hex_merged_wgs84 = hex_merged.to_crs(epsg=4326)

    centroid = hex_merged_wgs84.unary_union.centroid
    center = dict(lat=centroid.y, lon=centroid.x)

    fig = go.Figure()

    for _, row in hex_merged_wgs84.iterrows():
        poly = row.geometry
        gini_val = row.get(gini_col)
        mean_time = row.get("mean_time_min")

        xs, ys = poly.exterior.xy

        if pd.notna(gini_val) and np.isfinite(float(gini_val)):
            color = gini_to_color_hex(float(gini_val))
            gini_text = f"{float(gini_val):.4f}"
        else:
            color = "#CCCCCC"
            gini_text = "N/A"

        fillcol = _hex_to_rgba(color, 0.35)

        zone_id = row.get("id")
        try:
            zone_key = str(int(zone_id))
        except Exception:
            zone_key = str(zone_id) if zone_id is not None else None

        label = labels_map.get(zone_key) if zone_key is not None else None
        if not label:
            label = f"Zona {zone_key}" if zone_key is not None else "Zona ?"

        hover_text = (
            f"<b>{label}</b>"
            f"<br>Gini: {gini_text}"
            f"<br>Tempo medio: {fmt_min(mean_time)} min"
        )

        fig.add_trace(go.Scattermapbox(
            lon=list(xs),
            lat=list(ys),
            mode="lines",
            line=dict(color=color, width=2),
            fill="toself",
            fillcolor=fillcol,
            hovertext=hover_text,
            hoverinfo="text",
            showlegend=False,
        ))

    fig.update_layout(
        title=f"Mappa Gini per esagono ({gini_col})",
        margin=dict(l=0, r=0, t=50, b=0),
        mapbox=dict(
            style="carto-positron",
            center=center,
            zoom=11
        ),
        hovermode="closest",
        height=650,
    )
    return fig


In [10]:
# Cell 6 — Plot mappa (usa Gini "puro" 0..1: gini_time_norm)
fig_map = render_hexagon_heatmap(
    hexes_gdf=hexes_gdf,
    metrics_df=metrics_df,
    labels_map=center_labels,
    gini_col="gini_time_norm",   # <-- consigliato per bin 0.1
)
fig_map.show()


/var/folders/32/v2rvwmhs7cl5g_6749vpj90h0000gn/T/ipykernel_28800/528554471.py:53: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.

/var/folders/32/v2rvwmhs7cl5g_6749vpj90h0000gn/T/ipykernel_28800/528554471.py:90: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [9]:
# Cell 7 — Istogramma Plotly dei Gini per esagono (bin = 0.1)

gini_vals = pd.to_numeric(metrics_df["gini_time_norm"], errors="coerce").dropna()
gini_vals = gini_vals[(gini_vals >= 0) & (gini_vals <= 1)]

fig_hist = go.Figure(
    data=[
        go.Histogram(
            x=gini_vals,
            xbins=dict(start=0.0, end=1.0, size=0.1),
        )
    ]
)

fig_hist.update_layout(
    title="Distribuzione Gini per esagono (bin = 0.1)",
    xaxis_title="Gini (0..1)",
    yaxis_title="Conteggio esagoni",
    margin=dict(l=0, r=0, t=50, b=0),
    height=420,
)
fig_hist.show()


In [14]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon

def hex_polygon(cx, cy, radius_m):
    """Crea un esagono piatto centrato in (cx, cy) con raggio in metri."""
    angles = np.linspace(0, 2 * np.pi, 7)[:-1] + np.pi / 6
    return Polygon([(cx + radius_m * np.cos(a), cy + radius_m * np.sin(a)) for a in angles])

# Carica i nuovi centri
centers = gpd.read_file("./centri_esagoni_parigi.geojson").to_crs(epsg=2154)

# Scegli il raggio in metri (es. 400m per esagoni di circa 800m di diametro)
RADIUS_M = 250

centers["geometry"] = centers.apply(
    lambda r: hex_polygon(r.geometry.x, r.geometry.y, RADIUS_M), axis=1
)

# Salva
centers.to_crs(epsg=4326).to_file("esagoni_parigi.geojson", driver="GeoJSON")
print(f"Salvati {len(centers)} esagoni.")

Salvati 520 esagoni.
